All provided files should be in the same folder.

### Load packages

In [204]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import pandas_profiling
import datetime
%matplotlib inline
from io import StringIO
import os
import json
import re

### Load the file

In [205]:
# we can deal with it just in the memory. Alternatively I pushed it through Databricks and uploaded the final table into a Postgresql database. 
os.chdir(r'C:\Users\us61565\Desktop\Internal\Kaggle') # use the right address here

# Reading the json as a dict
with open('sc_data_science_challenge.json') as json_data:
    data = json.load(json_data)

In [ ]:
data

### Make a dataframe

In [207]:
# Had to make everything of an equal length first 
def pad_dict_list(dict_list, padel):
    lmax = 0
    for lname in dict_list.keys():
        lmax = max(lmax, len(dict_list[lname]))
    for lname in dict_list.keys():
        ll = len(dict_list[lname])
        if  ll < lmax:
            dict_list[lname] += [padel] * (lmax - ll)
    return dict_list

In [208]:
data1 = pad_dict_list(data, 1) # using the function from above

In [209]:
df = pd.DataFrame.from_dict(data1).T

In [210]:
cols = df.T.iloc[0:15,1]  # saving columns' names
cols

0                                                ts
1                                      country_code
2                                    client_version
3                                 listening_context
4                        recommender_algorithm_name
5                                          track_id
6                              track_genre_category
7                                 track_upload_date
8                                    track_duration
9                                   listen_duration
10                                      listener_id
11                             listener_signup_date
12             listener_top_genre_category_listened
13               listener_prev_month_listening_time
14    listener_prev_month_avg_daily_tracks_listened
Name: columns, dtype: object

In [211]:
cols_2_split = df.T.iloc[0:,0] # data to split

In [212]:
# make sure seconds are good. Remove from the final
original = cols_2_split.astype(str).str.split(', ', expand = True) 

In [213]:
original.columns = cols

In [214]:
original.head()

columns,ts,country_code,client_version,listening_context,recommender_algorithm_name,track_id,track_genre_category,track_upload_date,track_duration,listen_duration,listener_id,listener_signup_date,listener_top_genre_category_listened,listener_prev_month_listening_time,listener_prev_month_avg_daily_tracks_listened
0,[1467398683,'GB','204.0.20','tracks','content-based','b47911d0e80d1a8a959a2b726654bbfa','Dance & Electronic',1466640000,413933,413933,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
1,[1467398924,'GB','204.0.20','tracks','content-based','1f5e4bc82ba7fc802f780870f9b69191','HipHop & R&B',1466726400,402640,110722,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
2,[1467820636,'GB','204.0.20','users','content-based','092495283958e5d00f61cf2afe970d83','Pop',1467072000,494811,193818,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
3,[1467401951,'US','36.0.8','collection','content-based','211cd7be59941679f5ea847879b3b19b','HipHop & R&B',1402272000,361585,361585,'000264c5b34ad04999535c8b6a8b86e2',1379376000,'HipHop & R&B',112239180.0,10.5]
4,[1467402159,'US','36.0.8','collection','content-based','0e7ca1a7524aba84473540fa0d0ec75f','Pop',1448668800,373700,160162,'000264c5b34ad04999535c8b6a8b86e2',1379376000,'HipHop & R&B',112239180.0,10.5]


In [215]:
# remove
# t1 = cols_2_split.astype(str).map(lambda x: x[0]).str.rsplit(', ').str

In [216]:
df_values = cols_2_split.astype(str).str.split(', ', expand = True) 

In [217]:
# Giving names to columns
df_values.columns = cols

In [218]:
df_values.head()

columns,ts,country_code,client_version,listening_context,recommender_algorithm_name,track_id,track_genre_category,track_upload_date,track_duration,listen_duration,listener_id,listener_signup_date,listener_top_genre_category_listened,listener_prev_month_listening_time,listener_prev_month_avg_daily_tracks_listened
0,[1467398683,'GB','204.0.20','tracks','content-based','b47911d0e80d1a8a959a2b726654bbfa','Dance & Electronic',1466640000,413933,413933,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
1,[1467398924,'GB','204.0.20','tracks','content-based','1f5e4bc82ba7fc802f780870f9b69191','HipHop & R&B',1466726400,402640,110722,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
2,[1467820636,'GB','204.0.20','users','content-based','092495283958e5d00f61cf2afe970d83','Pop',1467072000,494811,193818,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
3,[1467401951,'US','36.0.8','collection','content-based','211cd7be59941679f5ea847879b3b19b','HipHop & R&B',1402272000,361585,361585,'000264c5b34ad04999535c8b6a8b86e2',1379376000,'HipHop & R&B',112239180.0,10.5]
4,[1467402159,'US','36.0.8','collection','content-based','0e7ca1a7524aba84473540fa0d0ec75f','Pop',1448668800,373700,160162,'000264c5b34ad04999535c8b6a8b86e2',1379376000,'HipHop & R&B',112239180.0,10.5]


In [219]:
# test = df_values

In [220]:
# test.head()

In [221]:
df_values.head()

columns,ts,country_code,client_version,listening_context,recommender_algorithm_name,track_id,track_genre_category,track_upload_date,track_duration,listen_duration,listener_id,listener_signup_date,listener_top_genre_category_listened,listener_prev_month_listening_time,listener_prev_month_avg_daily_tracks_listened
0,[1467398683,'GB','204.0.20','tracks','content-based','b47911d0e80d1a8a959a2b726654bbfa','Dance & Electronic',1466640000,413933,413933,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
1,[1467398924,'GB','204.0.20','tracks','content-based','1f5e4bc82ba7fc802f780870f9b69191','HipHop & R&B',1466726400,402640,110722,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
2,[1467820636,'GB','204.0.20','users','content-based','092495283958e5d00f61cf2afe970d83','Pop',1467072000,494811,193818,'0001d5755511cae40e5b32d83a3decce',1324771200,'HipHop & R&B',617270940.0,22.5]
3,[1467401951,'US','36.0.8','collection','content-based','211cd7be59941679f5ea847879b3b19b','HipHop & R&B',1402272000,361585,361585,'000264c5b34ad04999535c8b6a8b86e2',1379376000,'HipHop & R&B',112239180.0,10.5]
4,[1467402159,'US','36.0.8','collection','content-based','0e7ca1a7524aba84473540fa0d0ec75f','Pop',1448668800,373700,160162,'000264c5b34ad04999535c8b6a8b86e2',1379376000,'HipHop & R&B',112239180.0,10.5]


### Cleaning the dataset

In [222]:
# Square brackets
df_values['ts'] = df_values['ts'].str[1:] # removing the first bracket
df_values.iloc[:,-1] = df_values.iloc[:,-1].map(lambda x: str(x)[:-1])  # don't want to type the name of the last column. Removing the bracket

In [223]:
# removing quotes
for a in df_values.iloc[:,1:7].columns.tolist():
    df_values[a] = df_values[a].astype(str).str.replace('\'', '')

for b in df_values[['listener_id','listener_top_genre_category_listened']].columns.tolist():
    df_values[b] = df_values[b].astype(str).str.replace('\'', '')    

In [224]:
# unix timestamp to date
df_values['ts'] = pd.to_datetime(df_values['ts'], unit='s')
df_values['track_upload_date'] = pd.to_datetime(df_values['track_upload_date'], unit='s')
df_values['listener_signup_date'] = pd.to_datetime(df_values['listener_signup_date'], unit='s')

In [225]:
# Milliseconds to seconds for now
df_values['track_duration'] = df_values['track_duration'].astype(float)/1000
df_values['listen_duration'] = df_values['listen_duration'].astype(float)/1000

In [244]:
"""The above approach won't work with listener_prev_month_listening_time because it has a non-numeric value for customers who are 
first month users. Here is a work-around"""

df_values['listener_prev_month_listening_time'] = pd.to_numeric(df_values['listener_prev_month_listening_time'], errors='ignore')
df_values['listener_prev_month_listening_time'].replace(to_replace=['None'], value=np.nan, inplace=True)
df_values['listener_prev_month_listening_time'].fillna(0, inplace = True)
df_values['listener_prev_month_listening_time'] = df_values['listener_prev_month_listening_time'].astype(float)/1000

In [247]:
# let's check the NaN situation
df_values[df_values.isnull().any(axis=1)]

columns,ts,country_code,client_version,listening_context,recommender_algorithm_name,track_id,track_genre_category,track_upload_date,track_duration,listen_duration,listener_id,listener_signup_date,listener_top_genre_category_listened,listener_prev_month_listening_time,listener_prev_month_avg_daily_tracks_listened


In [248]:
df_values.dtypes

columns
ts                                               datetime64[ns]
country_code                                             object
client_version                                           object
listening_context                                        object
recommender_algorithm_name                               object
track_id                                                 object
track_genre_category                                     object
track_upload_date                                datetime64[ns]
track_duration                                          float64
listen_duration                                         float64
listener_id                                              object
listener_signup_date                             datetime64[ns]
listener_top_genre_category_listened                     object
listener_prev_month_listening_time                      float64
listener_prev_month_avg_daily_tracks_listened            object
dtype: object

In [ ]:
# writing a .csv
# df_values.to_csv(r'C:\Users\us61565\Desktop\Internal\Kaggle\df_values.csv', index=False)

## Descriptive Analysis

Seems it's much easier to take a first look using Tableau. 

First we need to define what a skipped track is:


<img src="skip_track_def.jpg">

Let's see whether this metric has been changing over years:

Then we move to assessing how every dimension (or a feature) influences 

In [ ]:
# cleaning columns from brackets, quotes, etc.
# df['result'] = df['result'].str.replace(r'\D', '')
# df_values['ts'] = df_values['ts'].str.replace(r'[','')
# df_values['ts'] = df_values['ts'].str[1:] # removing the first bracket
# df_values.iloc[:,-1] = df_values.iloc[:,-1].map(lambda x: str(x)[:-1])  # don't want to type the name of the last column. Removing the bracket
# df_values['country_code'].apply(lambda s: s.str.replace('\'', ''))
# df_values['client_version'].apply(lambda s: s.str.replace('\'', ''))
# df_values['listening_context'].apply(lambda s: s.str.replace('\'', ''))
# df_values['listening_context'].apply(lambda s: s.str.replace('\'', ''))
# df_values['listening_context'].apply(lambda s: s.str.replace('\'', ''))

In [ ]:
# test['country_code'].apply(lambda s: s.str.replace('\'', ''))
# test['country_code'].astype(str).str.replace('\'', '')

In [ ]:
# removing brackets:
# test['ts'] = test['ts'].str[1:]
# test.iloc[:,-1] = test.iloc[:,-1].map(lambda x: str(x)[:-1])  # don't want to type the name of the last column. Removing the bracket


# # removing quotes
# for a in test.iloc[:,1:7].columns.tolist():
#     test[a] = test[a].astype(str).str.replace('\'', '')

# for b in test[['listener_id','listener_top_genre_category_listened']].columns.tolist():
#     test[b] = test[b].astype(str).str.replace('\'', '')    


In [ ]:
# unix timestamp to date
# test['ts'] = pd.to_datetime(test['ts'], unit='s')
# test['track_upload_date'] = pd.to_datetime(test['track_upload_date'], unit='s')
# test['listener_signup_date'] = pd.to_datetime(test['listener_signup_date'], unit='s')


In [ ]:
# converting milliseconds to seconds for now
# test['track_duration'] = test['track_duration'].astype(float)/1000
# test['listen_duration'] = test['listen_duration'].astype(float)/1000
# test['listener_prev_month_listening_time'] = test['listener_prev_month_listening_time'].astype(float)/1000

In [ ]:
# test['listener_prev_month_listening_time'].astype(str).applymap(lambda x: isinstance(x, (int, float)))
# test['listener_prev_month_listening_time'] = test['listener_prev_month_listening_time'].astype(str)

In [ ]:
# test['listener_prev_month_listening_time'].astype(str).apply(lambda x: isinstance(x, (str)))

In [ ]:
# test['listener_prev_month_listening_time']

In [ ]:
# test.head()

In [ ]:
# patterns= [r'\D+'] # non integers
# # for p in patterns:
# #     match=  re.findall(p, test['listener_prev_month_listening_time'])
# #     print(match)

# # test['listener_prev_month_listening_time'].str.contains(patterns, regex=True)
# # test['listener_prev_month_listening_time'] = df_values['listener_prev_month_listening_time']
# test['listener_prev_month_listening_time'] = pd.to_numeric(test['listener_prev_month_listening_time'], errors='ignore')
# test['listener_prev_month_listening_time'].fillna(0, inplace = True)
# # test['listener_prev_month_listening_time'].apply(lambda x: any(pd.Series(x).str.contains(patterns)))

In [ ]:
# test[test.isnull().any(axis=1)]